## Objetivo do notebook
#### O objetivo deste notebook é realizar a leitura de dados das origens do dataset avaliações de turmas na camada gold  e gravar no DW

### Import Libs

In [1]:
!pip install minio

### Import das libs

In [2]:
from pyspark.sql import SparkSession
from minio import Minio, error
from io import BytesIO

import os
import pandas as pd
import pyarrow.parquet as pq

### Definição de variáveis

In [3]:
# Sessão Spark
spark = SparkSession.builder \
    .appName("CSV to PostgreSQL") \
    .config("spark.driver.extraClassPath", "/home/jovyan/notebooks/jars/postgresql-42.6.0.jar") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Parametros de input e output das origens
camadaLeitura= 'gold'
pasta = 'avaliacoes/'
temp_blobs = '/home/jovyan/notebooks/temporary_blobs/' # pasta temporária para armazenamento de objetos


# Conexão ao miniIO
minio_endpoint = 'minio:9000'
minio_access_key = 'minioaccesskey'
minio_secret_key = 'miniosecretkey'
minio_object_name = pasta
minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=False)

### Leitura da camada Gold

In [4]:
dataframes = []

try:
    # Listar objetos no bucket
    objects = minio_client.list_objects(camadaLeitura, prefix=pasta, recursive=True)

    # Iterar sobre os objetos na pasta específica
    for obj in objects:
        # Lê o conteúdo do objeto
        content = minio_client.get_object(camadaLeitura, obj.object_name).read()
        
        # Lê o conteúdo do objeto Parquet usando pyarrow e converte para DataFrame Pandas
        df_pandas = pq.read_table(BytesIO(content)).to_pandas()
        
        # Adiciona o DataFrame Pandas à lista de DataFrames
        dataframes.append(df_pandas)
        
        print(f"Objeto encontrado: {obj.object_name}")
except error.MinioException as e:
    print(f"Erro ao listar objetos: {e}")

# Concatena todos os DataFrames Pandas em um único DataFrame Pandas
consolidated_df_pandas = pd.concat(dataframes, ignore_index=True)

# Converte para DataFrame PySpark
df_avaliacoes = spark.createDataFrame(consolidated_df_pandas)

Objeto encontrado: avaliacoes/part-00000-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00001-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00002-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00003-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00004-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00005-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00006-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00007-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00008-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: avaliacoes/part-00009-3a09b3e5-6791-4916-a4f3-bcf64e4625a1-c000.snappy.parquet
Objeto encontrado: a

### Gravação do dataframe em um diretório temporário

In [5]:
df_avaliacoes.write.parquet(temp_blobs, mode="overwrite")

### Gravação no DW

In [6]:
df_avaliacoes = spark.read.parquet(temp_blobs, header=True)

postgres_url = "jdbc:postgresql://postgres:5432/prova"
postgres_properties = {
    "user": "postgres",
    "password": "password",
    "driver": "org.postgresql.Driver"
}
postgres_table = "fato_avaliacoes"

df_avaliacoes.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", postgres_table) \
    .option("user", postgres_properties["user"]) \
    .option("password", postgres_properties["password"]) \
    .option("driver", postgres_properties["driver"]) \
    .mode("overwrite") \
    .save()

print('Dados inseridos no banco de dados!')


Dados inseridos no banco de dados!


### Remove dados do diretório temporário de blobs

In [7]:
# Liste todos os arquivos na pasta
arquivos_na_pasta = os.listdir(temp_blobs)

# Itere sobre os arquivos e os delete
for arquivo in arquivos_na_pasta:
    caminho_completo = os.path.join(temp_blobs, arquivo)
    try:
        if os.path.isfile(caminho_completo):
            os.remove(caminho_completo)
            print(f'{caminho_completo} deletado com sucesso.')
    except Exception as e:
        print(f'Erro ao deletar {caminho_completo}: {e}')
        
spark.stop()

/home/jovyan/notebooks/temporary_blobs/.part-00000-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00001-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00002-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00003-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00004-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00005-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00006-712958a9-d44f-481c-9bd6-328d5d75d326-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00007-712958a9-d4